# Exploring San Diego Neighborhoods - to open an Indian Restaurant

# Introduction   
For many residents in San Diego, visiting Indian restaurants is a great way to relax and enjoy themselves or even to grab a gift for a friend. They have a wide variety of desserts to choose from. Some Indian restaurants are like a one-stop destination for all types of foodies. For Indian restaurant owners, the central location and the large crowds near Indian Restaurants provide a great distribution channel to market their products and services. Business developers are also taking advantage of this trend and are building more bars to cater to the demand. As a result, there are many Indian restaurants in San Diego and many more are being built. Opening a Indian restaurant allows the business owner to earn a consistent income. Of course, as with any business decision, opening a new location requires serious consideration and is a lot more complicated with a lot of moving parts. Particularly, the location of the Indian restaurant is one of the most important decisions that will determine whether the business will be a success or a failure.

# Sources of data and methods to extract them 

This wiki page (https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Diego) contains a list of neighborhoods in San Diego, with a total of 170 neighborhoods. I will be using web scraping techniques to extract the data from the wiki page, using Python requests and beautifulsoup packages. For the San Diego neighborhoods I will get the geographical coordinates of the neighborhoods using the Python Geocoder package which will give us the latitude and longitude coordinates of the neighborhoods.  After that, we will use Foursquare API to get the venue data for those neighborhoods. Foursquare has one of the largest databases of 105+ million places and is used by over 125,000 developers. The Foursquare API will provide many categories of the venue data, we are particularly interested in the desert bar category which will help us solve the business problem stated above. This is a project that will make use of the data science skills covered in this course. Skills such as working with APIs (Foursquare), data cleaning, data wrangling, machine learning (K-means clustering) and visualizing maps with folium. In the next section, we will present the Methodology section where we will discuss the steps taken in this project, the data analysis that we did and the machine learning techniques that were used. 

#  Methodology  
Firstly, we need to get the list of neighborhoods in the city of San Diego. Fortunately, the list is available in the Wikipedia page (https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Diego). We will use web scraping with Python requests and the beautifulsoup package to extract a list of the neighborhood’s data. However, this is just a list of names. We need to get the geographical coordinates in the form of latitude and longitude in order to be able to use Foursquare API. To do so, we will use the Geocoder package that will allow us to convert address into geographical coordinates in the form of latitude and longitude. After gathering the data, we will populate the data into a pandas DataFrame and then visualize the neighborhoods on a map using the Folium package. This allows us to perform a stare and compare to make sure that the geographical coordinates returned by the Geocoder package are correctly plotted in the city of San Diego.  
Next, we will use Foursquare API to get the top 100 venues that are within a radius of 2000 meters. We need to register a Foursquare Developer Account in order to obtain the Foursquare ID and Foursquare secret key. We then make API calls to Foursquare passing in the geographical coordinates of the neighborhoods in a Python loop. Foursquare will return the venue data in JSON format and we will extract the venue name, venue category, venue latitude and longitude. With the data, we can validate how many venues were returned for each neighborhood and examine how many unique categories can be created from all the returned venues. Then, we will analyze each neighborhood by grouping the rows by neighborhood and taking the mean of the frequency of occurrence of each venue category. By doing so, we are also preparing the data for use in clustering. Since we are analyzing the “Indian Restaurant” data, we will filter the “Indian Restaurant” as a venue category for the neighborhoods.  
Lastly, we will perform clustering on the data by using k-means clustering. K-means clustering algorithm identifies k number of centroids, and then allocates every data point to the nearest cluster, while keeping the centroids as small as possible. It is one of the simplest and most popular unsupervised machine learning algorithms and is particularly suited to solve the problem for this project. We will cluster the neighborhoods into 3 clusters based on their frequency of occurrence for “Indian Restaurant”. The results will allow us to identify which neighborhoods have higher concentration of Indian Restaurants while which neighborhoods have fewer number of Indian Restaurants. Based on the occurrence of Indian Restaurants in different neighborhoods, it will help us to answer the question as to which neighborhoods are most suitable to open new Indian Restaurant


# 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
%pip install -- geocoder
%pip install --upgrade geocoder
%pip uninstall geocoder # to get coordinates
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Note: you may need to restart the kernel to use updated packages.
Requirement already up-to-date: geocoder in c:\users\vishv\anaconda3\lib\site-packages (1.38.1)
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


Note: you may need to restart the kernel to use updated packages.
Libraries imported.


# 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Diego").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
sd_df = pd.DataFrame({"Neighborhood": neighborhoodList})

sd_df = sd_df.drop(0)
sd_df = sd_df.drop(1)
sd_df = sd_df.drop(2)

In [7]:
sd_df

,Neighborhood
3,"Allied Gardens, San Diego"
4,"Alta Vista, San Diego"
5,"Alvarado Estates, San Diego"
6,"Azalea Park, San Diego"
7,"Bankers Hill, San Diego"
8,"Barrio Logan, San Diego"
9,"Bay Terraces, San Diego"
10,"Bird Rock, San Diego"
11,"Birdland, San Diego"
12,"Black Mountain Ranch, San Diego"


In [8]:
sd_df.shape

(127, 1)

# 3. Get the geographical coordinates

In [9]:
import geocoder
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Diego, CA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in sd_df["Neighborhood"].tolist() ]

In [11]:
coords

[[32.79633000000007, -117.09450999999996],
 [32.693160000000034, -117.06778999999995],
 [32.77722977470506, -117.05725645116517],
 [32.73284000000007, -117.10775999999998],
 [32.72849000000008, -117.16141999999996],
 [32.69467000000003, -117.13807999999995],
 [32.69166000000007, -117.04083999999995],
 [32.813120000000026, -117.26510999999999],
 [32.793150000000026, -117.15431999999998],
 [32.99950000000007, -117.14514999999994],
 [32.73105000000004, -117.05379999999997],
 [32.73375974652356, -117.12734315331029],
 [32.97883842044885, -117.08523692633462],
 [32.715050000000076, -117.09302999999994],
 [32.749690000000044, -117.10714999999999],
 [32.777372655863424, -117.14773216733681],
 [32.83457000000004, -117.19462999999996],
 [32.74150000531939, -117.05027986630986],
 [32.72231709163447, -117.16740310892881],
 [32.71568000000008, -117.16170999999997],
 [32.718920000000026, -117.15652999999998],
 [32.78762000000006, -117.06416999999999],
 [32.948390000000074, -117.25991999999997],
 [3

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
sd_df['Latitude'] = df_coords['Latitude']
sd_df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(sd_df.shape)
sd_df

(127, 3)


,Neighborhood,Latitude,Longitude
3,"Allied Gardens, San Diego",32.732840,-117.107760
4,"Alta Vista, San Diego",32.728490,-117.161420
5,"Alvarado Estates, San Diego",32.694670,-117.138080
6,"Azalea Park, San Diego",32.691660,-117.040840
7,"Bankers Hill, San Diego",32.813120,-117.265110
8,"Barrio Logan, San Diego",32.793150,-117.154320
9,"Bay Terraces, San Diego",32.999500,-117.145150
10,"Bird Rock, San Diego",32.731050,-117.053800
11,"Birdland, San Diego",32.733760,-117.127343
12,"Black Mountain Ranch, San Diego",32.978838,-117.085237


In [15]:
sd_df = sd_df.drop(127)
sd_df = sd_df.drop(128)
sd_df = sd_df.drop(129)

In [16]:
# save the DataFrame as CSV file
sd_df.to_csv("sd_df.csv", index=False)

In [17]:
sd_df

,Neighborhood,Latitude,Longitude
3,"Allied Gardens, San Diego",32.732840,-117.107760
4,"Alta Vista, San Diego",32.728490,-117.161420
5,"Alvarado Estates, San Diego",32.694670,-117.138080
6,"Azalea Park, San Diego",32.691660,-117.040840
7,"Bankers Hill, San Diego",32.813120,-117.265110
8,"Barrio Logan, San Diego",32.793150,-117.154320
9,"Bay Terraces, San Diego",32.999500,-117.145150
10,"Bird Rock, San Diego",32.731050,-117.053800
11,"Birdland, San Diego",32.733760,-117.127343
12,"Black Mountain Ranch, San Diego",32.978838,-117.085237


# 4. Create a map of San Diego with neighborhoods superimposed on top

In [18]:
# get the coordinates for San Diego
address = 'San Diego, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 32.7174209, -117.1627714.


In [19]:
# create map of San Diego using latitude and longitude values
map_sd = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(sd_df['Latitude'], sd_df['Longitude'], sd_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sd)  
    
map_sd

In [20]:
# save the map as HTML file
map_sd.save('map_sd.html')

# 5. Use the Foursquare API to explore the neighborhoods

In [21]:
CLIENT_ID = 'JPXMPF5J0VWC3FN5H3JCEPLXHYNPSCPVI5SN1WUPKNRHZWJQ' # your Foursquare ID
CLIENT_SECRET = 'MMNVTEUF1G0X0FPQ45WTY4CJAAWQOCUYHN405WTEL2X1YTH5' # your Foursquare Secret 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JPXMPF5J0VWC3FN5H3JCEPLXHYNPSCPVI5SN1WUPKNRHZWJQ
CLIENT_SECRET:MMNVTEUF1G0X0FPQ45WTY4CJAAWQOCUYHN405WTEL2X1YTH5


Now, let's get the top 100 venues that are within a radius of 1000 meters.

In [22]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sd_df['Latitude'], sd_df['Longitude'], sd_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9765, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Allied Gardens, San Diego",32.73284,-117.10776,City Farmers Nursery,32.737851,-117.092571,Garden Center
1,"Allied Gardens, San Diego",32.73284,-117.10776,Nate's Garden Grill,32.737970,-117.092530,American Restaurant
2,"Allied Gardens, San Diego",32.73284,-117.10776,Chiquita's Mexican Restaurant,32.723732,-117.107230,Mexican Restaurant
3,"Allied Gardens, San Diego",32.73284,-117.10776,Thorn Street Brewery,32.739407,-117.125496,Brewery
4,"Allied Gardens, San Diego",32.73284,-117.10776,Pet Zone Tropical Fish,32.749700,-117.102950,Pet Store


Let's check how many venues were returned for each neighorhood

In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Allied Gardens, San Diego",73,73,73,73,73,73
"Alta Vista, San Diego",100,100,100,100,100,100
"Alvarado Estates, San Diego",93,93,93,93,93,93
"Azalea Park, San Diego",38,38,38,38,38,38
"Bankers Hill, San Diego",83,83,83,83,83,83
"Barrio Logan, San Diego",59,59,59,59,59,59
"Bay Terraces, San Diego",12,12,12,12,12,12
"Bird Rock, San Diego",81,81,81,81,81,81
"Birdland, San Diego",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 358 uniques categories.


In [26]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Garden Center', 'American Restaurant', 'Mexican Restaurant',
       'Brewery', 'Pet Store', 'Grocery Store', 'Coffee Shop', 'Dive Bar',
       'Park', 'Asian Restaurant', 'Juice Bar', 'Thrift / Vintage Store',
       'Latin American Restaurant', 'Taco Place', 'Sandwich Place',
       'Wings Joint', 'Restaurant', 'Vietnamese Restaurant',
       'Chinese Restaurant', 'Convenience Store', 'Pharmacy',
       'Video Game Store', 'Snack Place', 'Mobile Phone Shop',
       'Fast Food Restaurant', 'Bank', 'Discount Store', 'Pizza Place',
       'Liquor Store', 'Breakfast Spot', 'Shipping Store', 'Trail',
       'Intersection', 'Market', 'Ethiopian Restaurant',
       'New American Restaurant', 'Marijuana Dispensary',
       'Farmers Market', 'Baseball Field', 'Basketball Court',
       'Fried Chicken Joint', "Dentist's Office", 'Bus Station', 'Food',
       'Sushi Restaurant', 'Italian Restaurant', 'Soccer Field',
       'Deli / Bodega', 'Steakhouse', 'Lounge'], dtype=object)

In [27]:
# check if the results contain "dessert bar"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

# 6. Analyze Each Neighborhood

In [28]:
# one hot encoding
sd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sd_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sd_onehot.columns[-1]] + list(sd_onehot.columns[:-1])
sd_onehot = sd_onehot[fixed_columns]

print(sd_onehot.shape)
sd_onehot.head()

(9765, 359)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Cafeteria,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Port,Post Office,Print Shop,Pub,Public Bathroom,RV Park,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each categor

In [29]:
sd_grouped = sd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sd_grouped.shape)
sd_grouped.drop(columns=['ATM','Accessories Store'],inplace=True)
sd_grouped

(124, 359)


,Neighborhoods,Adult Boutique,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Cafeteria,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Port,Post Office,Print Shop,Pub,Public Bathroom,RV Park,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sa

In [30]:
len(sd_grouped[sd_grouped["Indian Restaurant"] > 0])

24

In [31]:
num_top_venues = 10

for hood in sd_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = sd_grouped[sd_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allied Gardens, San Diego----
                  venue  freq
0    Mexican Restaurant  0.11
1           Pizza Place  0.07
2         Grocery Store  0.07
3     Convenience Store  0.07
4          Intersection  0.05
5                  Park  0.03
6  Fast Food Restaurant  0.03
7           Coffee Shop  0.03
8           Wings Joint  0.03
9           Bus Station  0.03


----Alta Vista, San Diego----
                           venue  freq
0                    Zoo Exhibit  0.15
1             Italian Restaurant  0.09
2                    Coffee Shop  0.05
3                           Park  0.04
4  Vegetarian / Vegan Restaurant  0.03
5                           Café  0.03
6            American Restaurant  0.03
7                      Gastropub  0.02
8                 Ice Cream Shop  0.02
9                     Taco Place  0.02


----Alvarado Estates, San Diego----
                  venue  freq
0    Mexican Restaurant  0.11
1        Sandwich Place  0.06
2           Coffee Shop  0.05
3               B

                     venue  freq
0       Italian Restaurant  0.13
1              Coffee Shop  0.06
2                    Hotel  0.05
3                     Café  0.03
4                  Brewery  0.03
5       Mexican Restaurant  0.03
6                     Park  0.03
7  New American Restaurant  0.03
8               Steakhouse  0.02
9             Dessert Shop  0.02


----Grantville, San Diego----
                 venue  freq
0  American Restaurant  0.06
1          Coffee Shop  0.05
2   Mexican Restaurant  0.04
3        Grocery Store  0.03
4           Restaurant  0.03
5           Taco Place  0.03
6         Dessert Shop  0.03
7   Italian Restaurant  0.03
8              Gay Bar  0.03
9          Pizza Place  0.03


----Harbor Island, San Diego----
                   venue  freq
0  Vietnamese Restaurant  0.09
1      Convenience Store  0.08
2     Mexican Restaurant  0.06
3     Chinese Restaurant  0.05
4            Pizza Place  0.04
5             Taco Place  0.04
6          Grocery Store  0.04
7  

                  venue  freq
0    Mexican Restaurant  0.29
1          Intersection  0.06
2   Fried Chicken Joint  0.04
3         Grocery Store  0.04
4  Fast Food Restaurant  0.04
5  Gym / Fitness Center  0.04
6     Convenience Store  0.02
7                  Park  0.02
8        Hardware Store  0.02
9        Sandwich Place  0.02


----Morena, San Diego----
            venue  freq
0           Trail  0.12
1            Park  0.09
2      Astrologer  0.06
3   Deli / Bodega  0.06
4    Liquor Store  0.06
5  Hardware Store  0.03
6       Reservoir  0.03
7           Track  0.03
8      Taco Place  0.03
9  Breakfast Spot  0.03


----Mount Hope, San Diego----
                  venue  freq
0    Mexican Restaurant  0.09
1     Convenience Store  0.08
2  Fast Food Restaurant  0.05
3           Pizza Place  0.04
4              Pharmacy  0.03
5          Liquor Store  0.03
6                  Park  0.03
7           Gas Station  0.03
8        Ice Cream Shop  0.02
9                Bakery  0.02


----Mountain V

              venue  freq
0              Park  0.08
1             Trail  0.08
2        Astrologer  0.08
3      Liquor Store  0.04
4  Business Service  0.04
5  Football Stadium  0.04
6          Mountain  0.04
7    Breakfast Spot  0.04
8        Taco Place  0.04
9       Golf Course  0.04


----Roseville-Fleetridge, San Diego----
                          venue  freq
0                   Zoo Exhibit  0.31
1                           Zoo  0.08
2                       Exhibit  0.08
3                     Gift Shop  0.05
4  Theme Park Ride / Attraction  0.05
5          Fast Food Restaurant  0.05
6                Farmers Market  0.03
7                          Farm  0.03
8                Scenic Lookout  0.03
9       Fruit & Vegetable Store  0.03


----Sabre Springs, San Diego----
                  venue  freq
0        Clothing Store  0.17
1           Coffee Shop  0.04
2         Shopping Mall  0.03
3                  Café  0.03
4    Seafood Restaurant  0.03
5                 Hotel  0.03
6   Sport

                 venue  freq
0                Hotel  0.05
1          Coffee Shop  0.05
2                  Bar  0.05
3   Seafood Restaurant  0.05
4                 Park  0.05
5  American Restaurant  0.04
6       Sandwich Place  0.04
7                Beach  0.03
8     Sushi Restaurant  0.03
9               Resort  0.03




In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhoods'] = sd_grouped['Neighborhoods']

for ind in np.arange(venues_sorted.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(sd_grouped.iloc[ind, :], num_top_venues)

sd_grouped.head()

,Neighborhoods,Adult Boutique,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Cafeteria,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Port,Post Office,Print Shop,Pub,Public Bathroom,RV Park,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sa

In [34]:
sd_grouped.columns.values.tolist()

['Neighborhoods',
 'Adult Boutique',
 'Airport',
 'Airport Lounge',
 'Airport Service',
 'American Restaurant',
 'Amphitheater',
 'Aquarium',
 'Arcade',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Arts & Entertainment',
 'Asian Restaurant',
 'Astrologer',
 'Athletics & Sports',
 'Auto Dealership',
 'Auto Workshop',
 'Automotive Shop',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Court',
 'Bay',
 'Beach',
 'Beach Bar',
 'Beer Bar',
 'Beer Store',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Bike Trail',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Border Crossing',
 'Botanical Garden',
 'Boutique',
 'Bowling Alley',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Burrito Place',
 'Bus Station',
 'Bus Stop',
 'Business Service',
 'Butcher',
 'Cable Car',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 '

Create a new DataFrame for Shopping Mall data only

In [37]:
sd_Indian = sd_grouped[["Neighborhoods","Indian Restaurant"]]
sd_Indian


,Neighborhoods,Indian Restaurant
0,"Allied Gardens, San Diego",0.000000
1,"Alta Vista, San Diego",0.000000
2,"Alvarado Estates, San Diego",0.000000
3,"Azalea Park, San Diego",0.000000
4,"Bankers Hill, San Diego",0.000000
5,"Barrio Logan, San Diego",0.000000
6,"Bay Terraces, San Diego",0.000000
7,"Bird Rock, San Diego",0.000000
8,"Birdland, San Diego",0.000000
9,"Black Mountain Ranch, San Diego",0.010309


# 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [38]:
# set number of clusters
kclusters = 3

sd_clustering = sd_Indian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2])

In [39]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sd_merged = sd_Indian.copy()


# add clustering labels
sd_merged["Cluster Labels"] = kmeans.labels_

In [40]:
sd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sd_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,"Allied Gardens, San Diego",0.0,0
1,"Alta Vista, San Diego",0.0,0
2,"Alvarado Estates, San Diego",0.0,0
3,"Azalea Park, San Diego",0.0,0
4,"Bankers Hill, San Diego",0.0,0


In [41]:
# merge  San Diego with  San Diego_data to add latitude/longitude for each neighborhood
sd_merged = sd_merged.join(sd_df.set_index("Neighborhood"), on="Neighborhood")

print(sd_merged.shape)
sd_merged.head() # check the last columns!

(124, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,"Allied Gardens, San Diego",0.0,0,32.73284,-117.10776
1,"Alta Vista, San Diego",0.0,0,32.72849,-117.16142
2,"Alvarado Estates, San Diego",0.0,0,32.69467,-117.13808
3,"Azalea Park, San Diego",0.0,0,32.69166,-117.04084
4,"Bankers Hill, San Diego",0.0,0,32.81312,-117.26511


In [42]:
# sort the results by Cluster Labels
print(sd_merged.shape)
sd_merged.sort_values(["Cluster Labels"], inplace=True)
sd_merged.head()

(124, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,"Allied Gardens, San Diego",0.0,0,32.732840,-117.107760
88,"Rancho Bernardo, San Diego",0.0,0,32.739863,-117.076071
87,"Point Loma, San Diego",0.0,0,32.961600,-117.128460
86,"Point Loma Heights, San Diego",0.0,0,32.927180,-117.019600
84,"Palm City, San Diego",0.0,0,32.712310,-117.248250


Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sd_merged['Latitude'], sd_merged['Longitude'], sd_merged['Neighborhood'], sd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# 8. Examine Clusters

Clusters-0

In [44]:
sd_merged.loc[sd_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,"Allied Gardens, San Diego",0.0,0,32.732840,-117.107760
88,"Rancho Bernardo, San Diego",0.0,0,32.739863,-117.076071
87,"Point Loma, San Diego",0.0,0,32.961600,-117.128460
86,"Point Loma Heights, San Diego",0.0,0,32.927180,-117.019600
84,"Palm City, San Diego",0.0,0,32.712310,-117.248250
82,"Pacific Beach, San Diego",0.0,0,32.681300,-117.060050
81,"Otay Mesa, San Diego",0.0,0,32.584430,-117.084070
80,"Otay Mesa West, San Diego",0.0,0,32.964019,-117.192095
78,"Ocean View Hills, San Diego",0.0,0,32.567510,-116.962430
77,"Ocean Beach, San Diego",0.0,0,32.579230,-117.058050


Clusters-1

In [45]:
sd_merged.loc[sd_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
60,"Midtown, San Diego",0.080460,1,32.89361,-117.13340
101,"Shelter Island, San Diego",0.050000,1,32.90156,-117.18833
102,"Sherman Heights, San Diego",0.040816,1,32.89198,-117.19538


Clusters-2

In [46]:
sd_merged.loc[sd_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
63,"Miramar, San Diego",0.010753,2,32.779815,-117.097525
9,"Black Mountain Ranch, San Diego",0.010309,2,32.978838,-117.085237
44,"Jamacha, San Diego",0.010000,2,32.840850,-117.257800
109,Southeast San Diego,0.019608,2,32.728500,-117.250020
51,"Linda Vista, San Diego",0.010309,2,32.740200,-117.238660
39,"Grantville, San Diego",0.010000,2,32.749960,-117.165110
83,"Pacific Highlands Ranch, San Diego",0.013514,2,32.733062,-117.249529
73,"North City, San Diego",0.020000,2,32.757127,-117.219749
74,"North Park, San Diego",0.010000,2,32.744640,-117.247700
22,"Del Mar Heights, San Diego",0.010000,2,32.753660,-117.194480


Filter restaurants out of all venues
Let's filter venues and get only restaurants

# Observations:
Most of the Indian Restaurants are concentrated in the central area of San Diego, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has very low number to totally no Indian restaurants in the neighborhoods. This represents a great opportunity and high potential areas to open new Indian restaurants as there is very little to no competition from existing Indian restaurants. Meanwhile, Indian restaurants in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of Indian restaurant. From another perspective, this also shows that the oversupply of Indian restaurants mostly happened in the central area of San Diego, with the suburbs only have a few restaurants. Therefore, this research recommends that restaurant owners capitalize on these findings to open new Indian restaurants in neighborhoods in cluster 1 with little to no competition. Restaurant owners with unique selling points that stand out from the competition can also open new Indain restaurants in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of Indian restaurant and suffering from intense competition.